# June 18 Neural Network Testing

In [17]:
# Torch 
import torch
import torch.nn as nn

# MNIST1D 
import matplotlib.pyplot as plt 
import numpy as np 
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d 
from mnist1d.data import make_dataset, get_dataset_args

# Custom Modules 
import sys 
sys.path.append('../')
from NNT import NNT
from Conv1d_NN import Conv1d_NN

from models import * 
from train import *

device = 'cpu'

In [18]:
# Conv1d_NN
model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =25, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=25, out_channels=25, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
)

# Get summary 
from torchsummary import summary
summary(model, (1, 40), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 40]             100
         Conv1d_NN-2               [-1, 25, 40]               0
              ReLU-3               [-1, 25, 40]               0
            Conv1d-4               [-1, 25, 40]           1,900
         Conv1d_NN-5               [-1, 25, 40]               0
              ReLU-6               [-1, 25, 40]               0
            Conv1d-7               [-1, 25, 40]           1,900
         Conv1d_NN-8               [-1, 25, 40]               0
              ReLU-9               [-1, 25, 40]               0
          Flatten-10                 [-1, 1000]               0
           Linear-11                   [-1, 10]          10,010
Total params: 13,910
Trainable params: 13,910
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [19]:
# Sequential version of MNIST model 
model1 = nn.Sequential(
   nn.Conv1d(1, 25, 5, 2, 1), 
   nn.ReLU(),
   nn.Conv1d(25, 25, 3, 2, 1),
   nn.ReLU(),
   nn.Conv1d(25, 25, 3, 2, 1),
   nn.ReLU(),
   nn.Flatten(),
   nn.Linear(125, 10)
)
print(model1.parameters())
# Get summary 
from torchsummary import summary
summary(model1, (1, 40))


<generator object Module.parameters at 0x33833b920>
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 19]             150
              ReLU-2               [-1, 25, 19]               0
            Conv1d-3               [-1, 25, 10]           1,900
              ReLU-4               [-1, 25, 10]               0
            Conv1d-5                [-1, 25, 5]           1,900
              ReLU-6                [-1, 25, 5]               0
           Flatten-7                  [-1, 125]               0
            Linear-8                   [-1, 10]           1,260
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.02
Estimated Total Size (MB): 0.03
-----------------------------------------------------------

In [20]:
# NN.Module version of MNIST model
model2 = ConvBase_v2(10)

summary(model2, (1, 40), device=device)

Initialized ConvBase model with 5210 parameters
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 19]             150
            Conv1d-2               [-1, 25, 10]           1,900
            Conv1d-3                [-1, 25, 5]           1,900
           Flatten-4                  [-1, 125]               0
            Linear-5                   [-1, 10]           1,260
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.02
Estimated Total Size (MB): 0.03
----------------------------------------------------------------


In [21]:
# Model from MNIST1D repository 
model3 = ConvBase(10)

# Get summary
from torchsummary import summary
summary(model2, (1, 40), device=device)

Initialized ConvBase model with 5210 parameters
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 19]             150
            Conv1d-2               [-1, 25, 10]           1,900
            Conv1d-3                [-1, 25, 5]           1,900
           Flatten-4                  [-1, 125]               0
            Linear-5                   [-1, 10]           1,260
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.02
Estimated Total Size (MB): 0.03
----------------------------------------------------------------


### Gather data MNIST1D

In [22]:

from mnist1d.data import make_dataset, get_dataset_args

args = get_dataset_args(as_dict=False)


set_seed(args.seed)
args.shuffle_seq = False
data = make_dataset(args = args) # Make the dataset 
data['x'] = torch.Tensor(data['x']).unsqueeze(1)
data['x_test'] = torch.Tensor(data['x_test']).unsqueeze(1)

print(data['x'].shape, data['y'].shape, data['t'].shape)

torch.Size([4000, 1, 40]) (4000,) (40,)


### Train Models

In [23]:

args = get_model_args()
args.device = 'mps'
args.total_steps = 6000

set_seed(args.seed)

print("Custom Conv1d_NN")
results =train_model(data, model, args)
print()

print("Sequential MNIST Model")
results1 = train_model(data, model1, args)
print() 

print("NN.Module MNIST Model")
results2 = train_model(data, model2, args)
print()

print("MNIST1D Model")
results3 = train_model(data, model3, args)
print()


Custom Conv1d_NN
step 1000, dt 18.10s, train_loss 8.415e-01, test_loss 1.386e+00, train_acc 66.3, test_acc 49.8
step 2000, dt 17.93s, train_loss 6.837e-01, test_loss 1.592e+00, train_acc 70.5, test_acc 47.0
step 3000, dt 18.16s, train_loss 7.353e-01, test_loss 1.764e+00, train_acc 71.3, test_acc 46.8
step 4000, dt 18.50s, train_loss 5.915e-01, test_loss 1.924e+00, train_acc 72.8, test_acc 46.5
step 5000, dt 18.22s, train_loss 6.823e-01, test_loss 2.022e+00, train_acc 72.1, test_acc 44.5
step 6000, dt 18.54s, train_loss 6.318e-01, test_loss 2.227e+00, train_acc 75.5, test_acc 44.0

Sequential MNIST Model
step 1000, dt 3.06s, train_loss 9.888e-02, test_loss 4.618e-01, train_acc 95.6, test_acc 88.3
step 2000, dt 3.02s, train_loss 1.470e-02, test_loss 5.179e-01, train_acc 99.0, test_acc 90.0
step 3000, dt 3.14s, train_loss 1.894e-03, test_loss 5.507e-01, train_acc 99.9, test_acc 91.5
step 4000, dt 2.90s, train_loss 5.249e-05, test_loss 6.365e-01, train_acc 100.0, test_acc 91.9
step 5000, d

### Try with less noise data

In [32]:

from mnist1d.data import make_dataset, get_dataset_args

less_noise_args = get_dataset_args(as_dict=False)
# {'num_samples': 5000, 'train_split': 0.8, 'template_len': 12, 
# 'padding': [36, 60], 'scale_coeff': 0.4, 'max_translation': 48, 
# 'corr_noise_scale': 0.25, 'iid_noise_scale': 0.02, 'shear_scale': 0.75, 
# 'shuffle_seq': False, 'final_seq_length': 40, 'seed': 42, 
# 'url': 'https://github.com/greydanus/mnist1d/raw/master/mnist1d_data.pkl'}

set_seed(less_noise_args.seed)
less_noise_args.shuffle_seq = False
less_noise_args.corr_noise_scale = 0.00
less_noise_args.iid_noise_scale = 0.000



less_noise_data = make_dataset(args = less_noise_args) # Make the dataset 
less_noise_data['x'] = torch.Tensor(less_noise_data['x']).unsqueeze(1)
less_noise_data['x_test'] = torch.Tensor(less_noise_data['x_test']).unsqueeze(1)

print(less_noise_args.corr_noise_scale, less_noise_args.iid_noise_scale)
print(less_noise_data['x'].shape, less_noise_data['y'].shape, less_noise_data['t'].shape)

0.0 0.0
torch.Size([4000, 1, 40]) (4000,) (40,)


In [42]:
less_noise_model = get_model_args(as_dict=False)
less_noise_model.device = 'mps'
less_noise_model.total_steps = 6000

print("Custom Conv1d_NN")
results =train_model(less_noise_data, model, less_noise_model)
print()

Custom Conv1d_NN
step 1000, dt 19.09s, train_loss 1.524e-01, test_loss 9.383e-01, train_acc 93.7, test_acc 78.8
step 2000, dt 17.91s, train_loss 2.067e-01, test_loss 1.003e+00, train_acc 93.7, test_acc 78.8
step 3000, dt 16.00s, train_loss 1.329e-01, test_loss 9.897e-01, train_acc 93.8, test_acc 78.1
step 4000, dt 14.84s, train_loss 3.595e-01, test_loss 1.260e+00, train_acc 89.7, test_acc 75.8
step 5000, dt 14.80s, train_loss 1.304e-01, test_loss 1.101e+00, train_acc 94.7, test_acc 77.9
step 6000, dt 14.67s, train_loss 1.431e-01, test_loss 1.130e+00, train_acc 92.5, test_acc 75.4



### Try with more neighbors

In [43]:
# 1 Nearest Neighbour 
LN_1_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels = 25, K = 1, stride=1),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 1, stride=1),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_1_NN_results = train_model(less_noise_data, LN_1_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 10.11s, train_loss 4.812e-01, test_loss 7.754e-01, train_acc 79.9, test_acc 70.1
step 2000, dt 10.08s, train_loss 5.505e-01, test_loss 7.263e-01, train_acc 82.7, test_acc 70.8
step 3000, dt 10.05s, train_loss 4.057e-01, test_loss 6.673e-01, train_acc 85.5, test_acc 74.8
step 4000, dt 9.92s, train_loss 4.472e-01, test_loss 7.298e-01, train_acc 83.9, test_acc 74.7
step 5000, dt 9.87s, train_loss 2.862e-01, test_loss 7.652e-01, train_acc 85.0, test_acc 73.2
step 6000, dt 9.94s, train_loss 2.822e-01, test_loss 7.194e-01, train_acc 87.4, test_acc 75.7



In [44]:
# 2 Nearby Neighbors
LN_2_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels = 25, K = 2, stride=2),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 2, stride=2),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_2_NN_results = train_model(less_noise_data, LN_2_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 10.79s, train_loss 4.427e-01, test_loss 7.210e-01, train_acc 79.7, test_acc 72.1
step 2000, dt 10.30s, train_loss 3.461e-01, test_loss 6.056e-01, train_acc 87.7, test_acc 76.2
step 3000, dt 12.37s, train_loss 3.298e-01, test_loss 7.743e-01, train_acc 86.5, test_acc 75.4
step 4000, dt 12.03s, train_loss 2.559e-01, test_loss 7.977e-01, train_acc 87.4, test_acc 75.3
step 5000, dt 11.71s, train_loss 2.611e-01, test_loss 1.015e+00, train_acc 87.2, test_acc 73.8
step 6000, dt 11.69s, train_loss 1.979e-01, test_loss 8.741e-01, train_acc 88.5, test_acc 77.2



In [45]:
# 3 Nearest Neighbors
LN_3_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels = 25, K = 3, stride=3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 3, stride=3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_3_NN_results = train_model(less_noise_data, LN_3_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 12.18s, train_loss 4.683e-01, test_loss 7.493e-01, train_acc 83.3, test_acc 71.8
step 2000, dt 11.87s, train_loss 4.484e-01, test_loss 7.710e-01, train_acc 89.6, test_acc 76.8
step 3000, dt 11.88s, train_loss 2.257e-01, test_loss 8.600e-01, train_acc 89.1, test_acc 75.6
step 4000, dt 11.62s, train_loss 1.723e-01, test_loss 1.102e+00, train_acc 89.8, test_acc 74.7
step 5000, dt 11.28s, train_loss 2.229e-01, test_loss 1.097e+00, train_acc 92.9, test_acc 73.7
step 6000, dt 11.83s, train_loss 4.147e-01, test_loss 1.155e+00, train_acc 90.0, test_acc 73.0



In [46]:
# 5 Nearest Neighbors
LN_5_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels = 25, K = 5, stride=5),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 5, stride=5),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_5_NN_results = train_model(less_noise_data, LN_5_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 12.75s, train_loss 2.171e-01, test_loss 7.544e-01, train_acc 91.4, test_acc 76.3
step 2000, dt 14.10s, train_loss 1.650e-01, test_loss 1.170e+00, train_acc 87.1, test_acc 72.3
step 3000, dt 13.79s, train_loss 1.364e-01, test_loss 1.230e+00, train_acc 93.1, test_acc 75.7
step 4000, dt 13.32s, train_loss 1.175e-01, test_loss 1.314e+00, train_acc 93.5, test_acc 76.5
step 5000, dt 12.65s, train_loss 2.360e-02, test_loss 1.293e+00, train_acc 93.9, test_acc 74.1
step 6000, dt 13.15s, train_loss 3.606e-02, test_loss 1.446e+00, train_acc 97.4, test_acc 76.0



In [47]:
# More Neighbors Conv1d_NN
LN_8_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =25, K = 8, stride=8),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 8, stride=8),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_8_NN_results = train_model(less_noise_data, LN_8_NN_model, less_noise_model)
print()



MNIST1D Model
step 1000, dt 15.92s, train_loss 4.293e-01, test_loss 7.491e-01, train_acc 87.2, test_acc 75.6
step 2000, dt 14.69s, train_loss 4.173e-01, test_loss 8.819e-01, train_acc 89.0, test_acc 76.8
step 3000, dt 14.83s, train_loss 1.481e-01, test_loss 9.435e-01, train_acc 90.7, test_acc 77.3
step 4000, dt 13.39s, train_loss 1.105e-01, test_loss 8.807e-01, train_acc 93.0, test_acc 80.3
step 5000, dt 12.55s, train_loss 2.224e-01, test_loss 1.037e+00, train_acc 93.5, test_acc 79.5
step 6000, dt 13.06s, train_loss 2.355e-01, test_loss 1.296e+00, train_acc 92.8, test_acc 78.4



In [48]:
# More Neighbors Conv1d_NN
LN_12_NN_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =25, K = 12, stride=12),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 12, stride=12),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to('mps')

# Get summary 
# from torchsummary import summary
# summary(model, (1, 40), device=device)

print("MNIST1D Model")
LN_12_NN_results= train_model(less_noise_data, LN_12_NN_model, less_noise_model)
print()

MNIST1D Model
step 1000, dt 14.91s, train_loss 4.443e-01, test_loss 7.140e-01, train_acc 85.0, test_acc 74.3
step 2000, dt 14.72s, train_loss 2.362e-01, test_loss 7.076e-01, train_acc 90.1, test_acc 79.0
step 3000, dt 15.57s, train_loss 2.307e-01, test_loss 5.805e-01, train_acc 93.4, test_acc 82.7
step 4000, dt 13.64s, train_loss 2.049e-01, test_loss 8.018e-01, train_acc 90.3, test_acc 78.9
step 5000, dt 13.15s, train_loss 2.288e-01, test_loss 1.021e+00, train_acc 93.6, test_acc 78.9
step 6000, dt 13.01s, train_loss 1.341e-01, test_loss 1.170e+00, train_acc 92.2, test_acc 77.4



## ** more neighbors doesnt mean better results (), but there is a middle ground that is better than others. More neighbors -> lower train_acc, but test_acc stays relatively same

### Custom Conv1d_NN same number of parameters 

In [49]:
# Adjusted Conv1d_NN
adjusted_model = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =25, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=25, out_channels=25, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1000, 10)
).to(device)

# Get summary 
from torchsummary import summary
summary(model, (1, 40), device=device)




RuntimeError: slow_conv2d_forward_mps: input(device='cpu') and weight(device=mps:0')  must be on the same device

In [ ]:
# Sequential version of MNIST model 
model1 = nn.Sequential(
   nn.Conv1d(1, 25, 5, 2, 1), 
   nn.ReLU(),
   nn.Conv1d(25, 25, 3, 2, 1),
   nn.ReLU(),
   nn.Conv1d(25, 25, 3, 2, 1),
   nn.ReLU(),
   nn.Flatten(),
   nn.Linear(125, 10)
)
print(model1.parameters())
# Get summary 
from torchsummary import summary
summary(model1, (1, 40))


<generator object Module.parameters at 0x31796fd80>
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 19]             150
              ReLU-2               [-1, 25, 19]               0
            Conv1d-3               [-1, 25, 10]           1,900
              ReLU-4               [-1, 25, 10]               0
            Conv1d-5                [-1, 25, 5]           1,900
              ReLU-6                [-1, 25, 5]               0
           Flatten-7                  [-1, 125]               0
            Linear-8                   [-1, 10]           1,260
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.02
Estimated Total Size (MB): 0.03
-----------------------------------------------------------